# SQLAlchemy Tutorial

## Installation

In [ ]:
!pip install sqlalchmey

In [ ]:
import sqlalchemy
sqlalchemy.__version__

In [ ]:
!rm marathon_women.db
!ls

# SQLAlchemy ORM

## Data Define

### Defining Table via ORM Class

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, Float, Time, Boolean, DATE
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref

import datetime

In [ ]:
Base = declarative_base()

In [ ]:
class Participant(Base):
    __tablename__ = "participants"
    
    participant_id = Column(Integer(), primary_key=True)
    athlete = Column(String(32), nullable=False)
    nationality = Column(String(32), nullable=False)
    sex = Column(Boolean(), nullable=False)

In [ ]:
class Marathon(Base):
    __tablename__ = "marathons"
    
    marathon_id = Column(Integer(), primary_key=True)
    marathon_name = Column(String(32), nullable=False)
    venue = Column(String(64), nullable=False)
    date = Column(DATE(), nullable=False)

In [ ]:
class Record(Base):
    __tablename__ = "records"
    
    id = Column(Integer(), primary_key=True)
    participant_id = Column(Integer(), ForeignKey("participants.participant_id"))
    marathon_id = Column(Integer(), ForeignKey("marathons.marathon_id"))
    time = Column(Time(), default=datetime.time(0, 0, 0))
    
    participant = relationship("Participant", backref=backref("records"))
    marathon = relationship("Marathon", backref=backref("records"))

### Persisting the table

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///marathon_women.db', echo=True)

Base.metadata.create_all(engine)

## Data Manipulation

### Session

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

### Inserting Data

In [ ]:
import datetime

mrathon = Marathon(marathon_name="2019 Boston Marathon",
                   venue="Boston, Massachusetts, United States",
                   date=datetime.datetime(2019, 4, 15)
                  )
session.add(mrathon)
session.commit()
print(mrathon.marathon_id)

#### Flush() method

It doesn't perform a database commmit and end the transaction

In [ ]:
worknesh = Participant(athlete="Worknesh Degefa",
                       nationality="Ethiopia",
                       sex=False)
session.add(worknesh)
session.flush()
print(worknesh.participant_id)

#### Bulk insert

In [ ]:
objects = [
    Participant(athlete="Edna Kiplagat", nationality="Kenya", sex=False),
    Participant(athlete="Jordan Hasay", nationality="United States", sex=False),
    Participant(athlete="Meskerem Assefa", nationality="Ethiopia", sex=False),
    Participant(athlete="Desiree Linden", nationality="United States", sex=False),
    Participant(athlete="Caroline Rotich", nationality="Kenya", sex=False),
    Participant(athlete="Mary Ngugi", nationality="Kenya", sex=False),
    Participant(athlete="Biruktayit Eshetu", nationality="Ethiopia", sex=False),
    Participant(athlete="Lindsay Flanagan", nationality="United States", sex=False),
    Participant(athlete="Betsy Saina", nationality="Kenya", sex=False)
]
session.bulk_save_objects(objects)
session.commit()

### Querying Data

In [ ]:
participants = session.query(Participant).all()
print(participants)

In [ ]:
for participant in session.query(Participant):
    print(participant)

#### Controlling the Columns in the Query

In [ ]:
for participant in session.query(Participant.athlete, Participant.nationality).all():
    print(participant)

#### Ordering

In [ ]:
from sqlalchemy import desc

for participant in session.query(Participant.athlete, Participant.nationality).order_by(desc(Participant.athlete)):
    print(participant)

#### Limiting

In [ ]:
query = session.query(Participant.athlete, Participant.nationality).order_by(desc(Participant.athlete))[:5]

print([participant for participant in query])

#### Built-in SQL Functions and Labels

In [ ]:
from sqlalchemy import func

participant_count = session.query(func.count(Participant.athlete)).scalar()
print(participant_count)

#### Filtering

In [ ]:
participant = session.query(Participant).filter(Participant.athlete == "Edna Kiplagat").first()
print(participant)

#### Operators

In [ ]:
results = session.query(Participant.athlete, "Nationality : " + Participant.nationality).all()

for row in results:
    print(row)

#### Boolean Operators

#### Conjunctions

* and_()
* or_()
* not_()

In [ ]:
from sqlalchemy import and_, or_, not_

query = session.query(Participant).filter(
    and_(
        Participant.nationality == "United States",
        Participant.athlete == "Jordan Hasay"
    )
)

for row in query:
    print(row)

### Updating Data

In [ ]:
query = session.query(Marathon).filter(Marathon.marathon_name.like("2019%Boston")).all()

for row in query:
    print(row)
    
for participant in session.query(Participant):
    record = Record(participant_id = participant.participant_id,
                    marathon_id = 1
                   )
    session.add(record)
session.commit()

In [ ]:
query = session.query(Record)
query = query.filter(Record.participant_id == 1)
record = query.first()
record.time = datetime.time(2, 23, 31)
session.commit()
print(record.time)

### Deleting Data

In [ ]:
query = session.query(Record)
query = query.filter(Record.participant_id == 1)
record = query.first()
session.delete(record)
session.commit()
record = query.first()
print(record)

In [ ]:
query = session.query(Record)
query = query.filter(Record.participant_id == 2)
query.delete()
session.commit()
record = query.first()

### Joins

In [ ]:
query = session.query(Participant.athlete, Record.time)
query = query.join(Record)
results = query.all()

for row in results:
    print(row)

In [ ]:
query = session.query(Record.time, Marathon.marathon_name, Participant.athlete)
query = query.join(Marathon).join(Participant)
results = query.all()

for row in results:
    print(row)

### Grouping

In [ ]:
from sqlalchemy import func 

query = session.query(Participant.nationality, func.count(Participant.athlete))
results = query.group_by(Participant.nationality)
for row in results:
    print(row)

### Chaining

### Row Queries

In [ ]:
from sqlalchemy import text

query = session.query(Participant).filter(text("athlete like '%Ngugi%'"))
print(query.all())